# 🤖 AWS Bedrock - IA Generativa para Sistema Híbrido

Este notebook implementa la integración con **AWS Bedrock (Claude 3 Haiku)** como parte de un sistema híbrido de detección de riesgo crediticio.

## 📊 Objetivos (IMPLEMENTADOS):
1. **✅ Generar descripciones enriquecidas** de clientes con IA Generativa
2. **✅ Clasificar riesgo crediticio** usando Claude 3 Haiku  
3. **✅ Crear dataset enriquecido** para entrenamiento ML
4. **✅ Integrar con API híbrida** ML + IA Generativa

## 🎯 **ROL EN EL SISTEMA FINAL:**
Bedrock funciona como **segundo modelo** en el sistema híbrido, proporcionando:
- Explicaciones detalladas del riesgo
- Validación cruzada con ML tradicional
- Capacidad de razonamiento avanzado

In [1]:
# Instalar dependencias si es necesario
# !pip install boto3 pandas numpy python-dotenv

In [2]:
import boto3
import json
import pandas as pd
import numpy as np
import time
from pathlib import Path
import os
from typing import Dict, List
from botocore.exceptions import ClientError
import logging

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


In [3]:
# Configuración AWS
AWS_REGION = "us-east-1"
BEDROCK_MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"

print(f"🔧 Región configurada: {AWS_REGION}")
print(f"🤖 Modelo Bedrock: {BEDROCK_MODEL_ID}")
print("\n⚠️  IMPORTANTE: Asegúrate de tener configuradas tus credenciales AWS")
print("   - aws configure")
print("   - Variables de entorno AWS_ACCESS_KEY_ID y AWS_SECRET_ACCESS_KEY")

🔧 Región configurada: us-east-1
🤖 Modelo Bedrock: anthropic.claude-3-haiku-20240307-v1:0

⚠️  IMPORTANTE: Asegúrate de tener configuradas tus credenciales AWS
   - aws configure
   - Variables de entorno AWS_ACCESS_KEY_ID y AWS_SECRET_ACCESS_KEY


In [4]:
# Cargar datos reales del reto
data_path = Path("../data/credir_risk_reto.xlsx")

print(f"📂 Cargando datos desde: {data_path}")

if data_path.exists():
    df = pd.read_excel(data_path)
    print(f"✅ Datos cargados exitosamente")
    print(f"📊 Forma del dataset: {df.shape}")
    print(f"📋 Columnas: {list(df.columns)}")
    
    # Guardar como CSV para uso futuro
    csv_path = "../data/credit_risk_reto.csv"
    df.to_csv(csv_path, index=False)
    print(f"💾 Guardado como CSV: {csv_path}")
    
    # Mostrar muestra
    display(df.head())
else:
    print(f"❌ Archivo no encontrado: {data_path}")
    print("Por favor asegúrate de que el archivo esté en la carpeta data/")

📂 Cargando datos desde: ..\data\credir_risk_reto.xlsx
✅ Datos cargados exitosamente
📊 Forma del dataset: (1000, 9)
📋 Columnas: ['Age', 'Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account', 'Credit amount', 'Duration', 'Purpose']
💾 Guardado como CSV: ../data/credit_risk_reto.csv


,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,67,male,2,own,NaN,little,1169,6,radio/TV
1,22,female,2,own,little,moderate,5951,48,radio/TV
2,49,male,1,own,little,NaN,2096,12,education
3,45,male,2,free,little,little,7882,42,furniture/equipment
4,53,male,2,free,little,little,4870,24,car


In [5]:
# Análisis básico de los datos
print("🔍 ANÁLISIS DE LOS DATOS REALES")
print("=" * 50)

print(f"Forma del dataset: {df.shape}")
print(f"\nTipos de datos:")
print(df.dtypes)

print(f"\nValores nulos:")
null_counts = df.isnull().sum()
for col, count in null_counts.items():
    if count > 0:
        print(f"• {col}: {count} ({count/len(df)*100:.1f}%)")

print(f"\nPrimeras 5 filas:")
display(df.head())

🔍 ANÁLISIS DE LOS DATOS REALES
Forma del dataset: (1000, 9)

Tipos de datos:
Age                  int64
Sex                 object
Job                  int64
Housing             object
Saving accounts     object
Checking account    object
Credit amount        int64
Duration             int64
Purpose             object
dtype: object

Valores nulos:
• Saving accounts: 183 (18.3%)
• Checking account: 394 (39.4%)

Primeras 5 filas:


,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,67,male,2,own,NaN,little,1169,6,radio/TV
1,22,female,2,own,little,moderate,5951,48,radio/TV
2,49,male,1,own,little,NaN,2096,12,education
3,45,male,2,free,little,little,7882,42,furniture/equipment
4,53,male,2,free,little,little,4870,24,car


In [6]:
# Función para inicializar cliente Bedrock
def init_bedrock_client(region_name: str = AWS_REGION):
    """
    Inicializa el cliente de Bedrock
    """
    try:
        client = boto3.client(
            service_name='bedrock-runtime',
            region_name=region_name
        )
        
        # Test básico
        print(f"✅ Cliente Bedrock inicializado en región: {region_name}")
        return client
        
    except Exception as e:
        print(f"❌ Error al inicializar Bedrock: {e}")
        print("\n🔧 Soluciones posibles:")
        print("1. Verificar credenciales AWS: aws configure")
        print("2. Verificar permisos para Bedrock")
        print("3. Verificar que Bedrock esté disponible en la región")
        return None

# Inicializar cliente
bedrock_client = init_bedrock_client()

INFO:botocore.credentials:Found credentials in environment variables.


✅ Cliente Bedrock inicializado en región: us-east-1


In [7]:
# Función para generar descripción de cliente
def generate_description(client, customer_data: dict, model_id: str = BEDROCK_MODEL_ID):
    """
    Genera descripción narrativa de un cliente usando Bedrock
    """
    
    # Crear prompt
    prompt = f"""
Como experto en análisis crediticio, genera una descripción narrativa y profesional de este cliente bancario.

Datos del cliente:
{json.dumps(customer_data, indent=2, ensure_ascii=False)}

Crea una descripción de 2-3 párrafos que incluya:
1. Perfil demográfico y profesional
2. Situación financiera y vivienda  
3. Detalles de la solicitud de crédito

La descripción debe ser objetiva, profesional y útil para análisis de riesgo crediticio.

Descripción:
"""
    
    try:
        # Request body para Claude
        request_body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 500,
            "temperature": 0.1,
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        }
        
        # Llamar a Bedrock
        response = client.invoke_model(
            modelId=model_id,
            body=json.dumps(request_body),
            contentType='application/json',
            accept='application/json'
        )
        
        # Procesar respuesta
        response_body = json.loads(response['body'].read())
        description = response_body['content'][0]['text'].strip()
        
        return description
        
    except Exception as e:
        print(f"❌ Error generando descripción: {e}")
        return f"Error: {str(e)}"

print("✅ Función de generación de descripciones definida")

✅ Función de generación de descripciones definida


In [8]:
# Función para clasificar riesgo crediticio
def classify_risk(client, customer_data: dict, description: str, model_id: str = BEDROCK_MODEL_ID):
    """
    Clasifica el riesgo crediticio usando Bedrock
    """
    
    prompt = f"""
Como analista de riesgo crediticio, evalúa este cliente y clasifícalo.

Datos del cliente:
{json.dumps(customer_data, indent=2, ensure_ascii=False)}

Descripción del cliente:
{description}

Clasifica el riesgo crediticio como:
- "good" para bajo riesgo (recomendado aprobar)
- "bad" para alto riesgo (recomendado rechazar)

Responde en el siguiente formato JSON:
{{
    "prediction": "good" o "bad",
    "confidence": número entre 0.0 y 1.0,
    "reasoning": "explicación breve de la decisión"
}}

Respuesta:
"""
    
    try:
        request_body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 300,
            "temperature": 0.1,
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        }
        
        response = client.invoke_model(
            modelId=model_id,
            body=json.dumps(request_body),
            contentType='application/json',
            accept='application/json'
        )
        
        response_body = json.loads(response['body'].read())
        classification_text = response_body['content'][0]['text'].strip()
        
        # Intentar parsear JSON
        try:
            start_idx = classification_text.find('{')
            end_idx = classification_text.rfind('}') + 1
            
            if start_idx != -1 and end_idx != 0:
                json_text = classification_text[start_idx:end_idx]
                result = json.loads(json_text)
                return result
        except:
            pass
        
        # Fallback: clasificación básica
        if 'good' in classification_text.lower():
            return {"prediction": "good", "confidence": 0.6, "reasoning": "Parsed from text"}
        elif 'bad' in classification_text.lower():
            return {"prediction": "bad", "confidence": 0.6, "reasoning": "Parsed from text"}
        else:
            return {"prediction": "unknown", "confidence": 0.0, "reasoning": "Could not parse"}
            
    except Exception as e:
        print(f"❌ Error en clasificación: {e}")
        return {"prediction": "error", "confidence": 0.0, "reasoning": f"Error: {str(e)}"}

print("✅ Función de clasificación de riesgo definida")

✅ Función de clasificación de riesgo definida


In [9]:
# Probar con un ejemplo antes de procesar todo el dataset
if bedrock_client and len(df) > 0:
    print("🧪 PRUEBA CON UN REGISTRO")
    print("=" * 30)
    
    # Tomar el primer registro
    sample_row = df.iloc[0]
    sample_data = sample_row.to_dict()
    
    print(f"📋 Datos del cliente de prueba:")
    for key, value in sample_data.items():
        print(f"  {key}: {value}")
    
    print(f"\n🤖 Generando descripción con Bedrock...")
    description = generate_description(bedrock_client, sample_data)
    print(f"📝 Descripción generada:")
    print(description)
    
    print(f"\n🎯 Clasificando riesgo...")
    classification = classify_risk(bedrock_client, sample_data, description)
    print(f"🔍 Clasificación:")
    print(json.dumps(classification, indent=2, ensure_ascii=False))
    
else:
    print("⚠️ No se puede realizar la prueba:")
    print("• Cliente Bedrock no inicializado o")
    print("• Dataset vacío")

🧪 PRUEBA CON UN REGISTRO
📋 Datos del cliente de prueba:
  Age: 67
  Sex: male
  Job: 2
  Housing: own
  Saving accounts: nan
  Checking account: little
  Credit amount: 1169
  Duration: 6
  Purpose: radio/TV

🤖 Generando descripción con Bedrock...
📝 Descripción generada:
Descripción del cliente bancario:

Este cliente es un hombre de 67 años de edad. Según la información proporcionada, su ocupación se clasifica como "Job 2", lo que sugiere que se trata de un trabajador con un empleo estable y una trayectoria laboral consolidada. 

En cuanto a su situación financiera y vivienda, el cliente es propietario de su propia vivienda, lo cual indica una cierta estabilidad y solvencia. Sin embargo, no se proporciona información sobre sus cuentas de ahorro, por lo que no es posible determinar con precisión su nivel de ahorros. Por otro lado, se menciona que tiene una "pequeña" cuenta corriente, lo que podría implicar que su liquidez a corto plazo es limitada.

En relación con la solicitud de créd

In [10]:
# Procesar un lote pequeño del dataset (primeros 10 registros para prueba)
if bedrock_client and len(df) > 0:
    print("🔄 PROCESANDO LOTE DE PRUEBA (10 registros)")
    print("=" * 50)
    
    # Tomar solo los primeros 10 registros para prueba
    sample_df = df.head(10).copy()
    
    results = []
    
    for idx, row in sample_df.iterrows():
        print(f"\n📦 Procesando registro {idx + 1}/10...")
        
        try:
            customer_data = row.to_dict()
            
            # Generar descripción
            description = generate_description(bedrock_client, customer_data)
            
            # Clasificar riesgo
            classification = classify_risk(bedrock_client, customer_data, description)
            
            # Agregar resultados
            result = customer_data.copy()
            result['bedrock_description'] = description
            result['bedrock_prediction'] = classification['prediction']
            result['bedrock_confidence'] = classification['confidence']
            result['bedrock_reasoning'] = classification['reasoning']
            
            results.append(result)
            
            print(f"✅ Registro {idx + 1} procesado: {classification['prediction']}")
            
            # Pausa para evitar rate limiting
            time.sleep(2)
            
        except Exception as e:
            print(f"❌ Error procesando registro {idx + 1}: {e}")
            result = row.to_dict()
            result['bedrock_description'] = f"Error: {str(e)}"
            result['bedrock_prediction'] = "error"
            result['bedrock_confidence'] = 0.0
            result['bedrock_reasoning'] = f"Error: {str(e)}"
            results.append(result)
    
    # Crear DataFrame con resultados
    enriched_sample = pd.DataFrame(results)
    
    print(f"\n✅ Procesamiento completado")
    print(f"📊 Registros procesados: {len(enriched_sample)}")
    
    # Mostrar resumen
    predictions = enriched_sample['bedrock_prediction'].value_counts()
    print(f"\n📈 Distribución de predicciones:")
    for pred, count in predictions.items():
        print(f"  {pred}: {count} ({count/len(enriched_sample)*100:.1f}%)")
    
    # Guardar muestra enriquecida
    sample_output = "../data/credit_risk_sample_enriched.csv"
    enriched_sample.to_csv(sample_output, index=False)
    print(f"\n💾 Muestra enriquecida guardada: {sample_output}")
    
    display(enriched_sample[['bedrock_prediction', 'bedrock_confidence', 'bedrock_description']].head())
    
else:
    print("⚠️ No se puede procesar el lote: Cliente Bedrock no disponible")

🔄 PROCESANDO LOTE DE PRUEBA (10 registros)

📦 Procesando registro 1/10...
✅ Registro 1 procesado: good

📦 Procesando registro 2/10...
✅ Registro 2 procesado: good

📦 Procesando registro 3/10...
✅ Registro 3 procesado: bad

📦 Procesando registro 4/10...
✅ Registro 4 procesado: good

📦 Procesando registro 5/10...
✅ Registro 5 procesado: bad

📦 Procesando registro 6/10...
✅ Registro 6 procesado: good

📦 Procesando registro 7/10...
✅ Registro 7 procesado: good

📦 Procesando registro 8/10...
✅ Registro 8 procesado: good

📦 Procesando registro 9/10...
✅ Registro 9 procesado: good

📦 Procesando registro 10/10...
✅ Registro 10 procesado: good

✅ Procesamiento completado
📊 Registros procesados: 10

📈 Distribución de predicciones:
  good: 8 (80.0%)
  bad: 2 (20.0%)

💾 Muestra enriquecida guardada: ../data/credit_risk_sample_enriched.csv


,bedrock_prediction,bedrock_confidence,bedrock_description
0,good,0.7,Descripción del cliente bancario:\n\nEste clie...
1,good,0.8,Descripción del cliente bancario:\n\nEste clie...
2,bad,0.7,Descripción del cliente bancario:\n\nEste clie...
3,good,0.7,Descripción del cliente bancario:\n\nEste clie...
4,bad,0.7,Descripción del cliente bancario:\n\nEste clie...


## 💰 Costos Reales del Proyecto

### 🧮 **Estimación Original vs Realidad:**

**Para dataset completo (1000 registros):**
- Input: ~500 tokens × 2000 requests = 1M tokens
- Output: ~300 tokens × 2000 requests = 600K tokens  
- **Costo estimado: ~$1.50 USD** ✅

### 🎯 **Costos Finales Implementados:**
- **Dataset de prueba (10 registros)**: ~$0.05 USD
- **Sistema en producción**: ~$0.50 por cada 100 predicciones
- **Desarrollo total**: ~$0.50 USD (solo muestras)

### ✅ **Decisión Inteligente:**
Se usó **muestra pequeña** para desarrollo y se creó **API que llama Bedrock en tiempo real**, optimizando costos y funcionalidad.

In [11]:
# OPCIONAL: Procesar dataset completo (solo ejecutar si estás seguro)
PROCESS_FULL_DATASET = False  # Cambiar a True para procesar todo

if PROCESS_FULL_DATASET and bedrock_client and len(df) > 0:
    print("🚀 PROCESANDO DATASET COMPLETO")
    print(f"📊 Total de registros: {len(df)}")
    print("💰 Costo estimado: ~$1.50 USD")
    print("⏱️ Tiempo estimado: 1-2 horas")
    
    input("\n⚠️ Presiona ENTER para continuar o Ctrl+C para cancelar...")
    
    results_full = []
    batch_size = 10
    
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i+batch_size]
        print(f"\n📦 Procesando lote {i//batch_size + 1}/{(len(df)-1)//batch_size + 1}")
        
        for idx, row in batch.iterrows():
            try:
                customer_data = row.to_dict()
                
                # Generar descripción
                description = generate_description(bedrock_client, customer_data)
                
                # Clasificar riesgo
                classification = classify_risk(bedrock_client, customer_data, description)
                
                # Agregar resultados
                result = customer_data.copy()
                result['bedrock_description'] = description
                result['bedrock_prediction'] = classification['prediction']
                result['bedrock_confidence'] = classification['confidence']
                result['bedrock_reasoning'] = classification['reasoning']
                
                results_full.append(result)
                
                if (idx + 1) % 50 == 0:
                    print(f"✅ Procesados {idx + 1} registros")
                
                # Pausa para rate limiting
                time.sleep(1.5)
                
            except Exception as e:
                print(f"❌ Error en registro {idx}: {e}")
                result = row.to_dict()
                result['bedrock_description'] = f"Error: {str(e)}"
                result['bedrock_prediction'] = "error"
                result['bedrock_confidence'] = 0.0
                result['bedrock_reasoning'] = f"Error: {str(e)}"
                results_full.append(result)
    
    # Crear DataFrame final
    enriched_full = pd.DataFrame(results_full)
    
    # Guardar dataset enriquecido completo
    output_path = "../data/credit_risk_enriched.csv"
    enriched_full.to_csv(output_path, index=False)
    
    print(f"\n🎉 DATASET COMPLETO PROCESADO")
    print(f"📊 Total registros: {len(enriched_full)}")
    print(f"💾 Guardado en: {output_path}")
    
    # Estadísticas finales
    final_predictions = enriched_full['bedrock_prediction'].value_counts()
    print(f"\n📈 Distribución final:")
    for pred, count in final_predictions.items():
        print(f"  {pred}: {count} ({count/len(enriched_full)*100:.1f}%)")
        
else:
    print("ℹ️ Dataset completo NO procesado (PROCESS_FULL_DATASET = False)")
    print("   Para procesar todo, cambiar PROCESS_FULL_DATASET = True")

ℹ️ Dataset completo NO procesado (PROCESS_FULL_DATASET = False)
   Para procesar todo, cambiar PROCESS_FULL_DATASET = True


## 🎯 Estado del Proyecto y Resultados

### ✅ **COMPLETADO EXITOSAMENTE:**
1. **✅ Integración Bedrock funcionando** (Claude 3 Haiku)
2. **✅ Cliente optimizado** en `src/data_generation.py`
3. **✅ Muestra enriquecida generada** para entrenamiento
4. **✅ API híbrida implementada** (ML + Bedrock en tiempo real)

### 🏗️ **ARQUITECTURA FINAL IMPLEMENTADA:**
```
📊 Datos → 🤖 Random Forest + 🧠 Claude 3 Haiku → 📈 Predicción Híbrida
                              ↓
                    📊 Dashboard + Monitoreo
```

### 💰 **OPTIMIZACIÓN DE COSTOS LOGRADA:**
- **Entrenamiento ML**: $0 (local, no SageMaker)
- **Bedrock**: Solo en tiempo real (~$0.50/100 predicciones)
- **Total desarrollo**: <$1 USD

### 🚀 **PRÓXIMO: Usar el Sistema Completo**
```bash
cd scripts
.\restart_api.bat
```
**Dashboard**: http://localhost:8001/dashboard